# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Load datasets

In [2]:
raw_LSOAs = gpd.read_file("datasets/cleaned/lsoa/cleaned_lsoa.gpkg")
raw_os_greenspace = gpd.read_file("datasets/cleaned/os_greenspace/cleaned_os_greenspace.gpkg")
raw_ua_greenspace = gpd.read_file("datasets/cleaned/ua_greenspace/cleaned_ua_greenspace.gpkg")

In [3]:
LSOAs = raw_LSOAs.copy()
os_greenspace = raw_os_greenspace.copy()
ua_greenspace = raw_ua_greenspace.copy()

# Combine greenspace datasets

In [4]:
# Combine greenspace datasets
combined_greenspace = pd.concat([
    ua_greenspace[['geometry']], 
    os_greenspace[['geometry']]
], ignore_index=True)

# Merge overlapping areas while keeping separate areas separate
unioned = combined_greenspace.geometry.union_all()
raw_combined_greenspace = gpd.GeoDataFrame(geometry=[unioned], crs=ua_greenspace.crs)
raw_combined_greenspace = raw_combined_greenspace.explode(index_parts=False).reset_index(drop=True)

In [5]:
combined_greenspace = raw_combined_greenspace.copy()

# Overlay LSOAs onto greenspace

In [6]:
LSOAs["total_area"] = LSOAs.geometry.area
overlay = gpd.overlay(combined_greenspace, LSOAs, how = "intersection")
LSOA_greenspace = overlay.groupby('lsoa')['geometry'].apply(lambda x: x.area.sum())
LSOAs['greenspace_area'] = LSOAs['lsoa'].map(LSOA_greenspace).fillna(0)
LSOAs['greenspace_proportion'] = LSOAs['greenspace_area'] / LSOAs["total_area"].replace(0, np.nan)

# Save output

In [8]:
LSOAs.to_file("datasets/combined/lsoa_greenspace.gpkg", driver="GPKG")